In [ ]:
!pip install contextily
!pip install pyrsgis

In [ ]:
import contextily as cx
from ipywidgets import interact
from math import floor
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, MaxNLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import pandas as pd
from pyrsgis import raster
from pyrsgis.convert import array_to_table
from pyrsgis.ml import imageChipsFromArray
import random
import seaborn as sns
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from statistics import mean
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# file names
feature_file = '/content/drive/MyDrive/Nadi_data_new/2013_original.tif'
# read the image
dsFeatures, arrFeatures = raster.read(feature_file, bands='all')
min_x = dsFeatures.bbox[0][0]
max_x = dsFeatures.bbox[1][0]
min_y = dsFeatures.bbox[0][1]
max_y = dsFeatures.bbox[1][1]

extent = [min_x, max_x, min_y, max_y]

In [ ]:
extent

In [ ]:
arrFeatures.shape

In [ ]:
import copy
from copy import deepcopy
from sklearn.feature_extraction import image

In [ ]:
def imageChipsFromSingleBandArray(data_arr, y_size=5, x_size=5):
    image_chips = deepcopy(data_arr)
    image_chips = np.pad(image_chips, (int(y_size/2),int(x_size/2)), 'reflect')
    image_chips = image.extract_patches_2d(image_chips, (y_size, x_size))

    return(image_chips)
def imageChipsFromArray_update(data_array, x_size=5, y_size=5):

    # if array is a single band image
    if len(data_array.shape) == 2:
        return(imageChipsFromSingleBandArray(data_array, x_size=x_size, y_size=y_size))

    # if array is a multi band image
    elif len(data_array.shape) > 2:
        data_array = copy.copy(data_array)
        data_array = np.rollaxis(data_array, 0, 3)

        for band in range(data_array.shape[2]):
            temp_array = imageChipsFromSingleBandArray(data_array[:, :, band], x_size=x_size, y_size=y_size)

            if band == 0:
                out_array = np.expand_dims(temp_array, axis=3)
            else:
                out_array = np.concatenate((out_array, np.expand_dims(temp_array, axis=3)), axis=3)

        return(out_array)

    # if shape of the image is less than two dimensions, raise error
    else:
        raise Exception("Sorry, only two or three dimensional arrays allowed.")

In [ ]:
# normalize the image and generate chips
arrFeatures_fuzzy = np.zeros(arrFeatures.shape)

for i in range(arrFeatures.shape[0]):
    bandMin = arrFeatures[i, :, :].min()
    bandMax = arrFeatures[i, :, :].max()
    bandRange = bandMax-bandMin
    for j in range(arrFeatures.shape[1]):
        for k in range(arrFeatures.shape[2]):
            arrFeatures_fuzzy[i, j, k] = (arrFeatures[i, j, k]-bandMin)/bandRange

#create chips using pyrsgis
#features_chips = imageChipsFromArray(arrFeatures_fuzzy, x_size=7, y_size=7)
features_chips = imageChipsFromArray_update(arrFeatures_fuzzy, x_size=9, y_size=9)

In [ ]:
features_chips.shape

In [ ]:
features_chips=features_chips.reshape(638040,9*9*10)

In [ ]:
features_chips.shape

*****

**2013_Training datast**

In [ ]:
positiveLabel_file = '/content/drive/MyDrive/Nadi_data_new/Label_2013_Nadi_1750_samples.tif'
dsPositiveLabels, arrPositiveLabels = raster.read(positiveLabel_file,bands='all')
n_class_pos = len(np.unique(arrPositiveLabels))

In [ ]:
n_class_pos

In [ ]:
arrPositiveLabels=arrPositiveLabels+1
np.unique(arrPositiveLabels)

In [ ]:
# generate random samples
nonZero_count = np.count_nonzero(arrPositiveLabels)
index = np.transpose(np.where(arrPositiveLabels==0))
index = index.astype(int)
randomIndex = random.sample(range(index.shape[0]), int(nonZero_count/(n_class_pos-1)))
arrNegativeLabels = np.zeros(arrPositiveLabels.shape)

for i in range(len(randomIndex)):
    arrNegativeLabels[index[randomIndex[i], 0], index[randomIndex[i], 1]] = n_class_pos # max value of labels plus one which equals n_class

arrPositiveLabels_flat = arrPositiveLabels.flatten()
arrNegativeLabels_flat = arrNegativeLabels.flatten()
# separate and balance the classes
positiveFeatures = features_chips[arrPositiveLabels_flat!=0]
positiveLabels = arrPositiveLabels_flat[arrPositiveLabels_flat!=0]

features=positiveFeatures
labels=positiveLabels

In [ ]:
len(features_chips[arrPositiveLabels_flat==6])

In [ ]:
temp = 250-91

In [ ]:
from sklearn.utils import resample
coastal_upsample = resample(features_chips[arrPositiveLabels_flat==6],
             replace=True,
             n_samples=temp,
             random_state=42)

In [ ]:
coastal_upsample.shape

In [ ]:
arr = np.concatenate((features, coastal_upsample), axis=0)

In [ ]:
features=arr

In [ ]:
coastal_label=[6]*temp

In [ ]:
type(coastal_label)

In [ ]:
new_label=np.concatenate((labels,coastal_label),axis=0)

In [ ]:
labels=new_label

In [ ]:
np.unique(labels)

In [ ]:
labels=labels-1

In [ ]:
n_class_pos = len(np.unique(labels))
print(n_class_pos)

In [ ]:
labels_df=pd.DataFrame()
labels_df['labels']=labels
labels_df['labels'].value_counts()

****

**Split**

In [ ]:
# # define a function to split features and labels
def train_test_split(features, labels, n_class_pos=n_class_pos, trainProp=0.7):
    labels_df=pd.DataFrame()
    labels_df['labels']=labels
    labels_df.reset_index(inplace=True)
    labels_df['labels']=labels_df['labels'].astype(int)

    train_list=[]
    test_list=[]
    for class_index in range(n_class_pos):
      sub_labels_df=labels_df[labels_df['labels'] == class_index ]
      dataSize=len(sub_labels_df['index'])
      sliceIndex = int(dataSize*trainProp)
      randIndex = list(sub_labels_df['index'])
      #random.seed(45)
      random.shuffle(randIndex)

      sub_train_list=randIndex[:sliceIndex]
      train_list = train_list + sub_train_list

      sub_test_list=randIndex[sliceIndex:]
      test_list = test_list + sub_test_list

    train_x = features[[train_list], :][0]
    test_x = features[[test_list], :][0]
    train_y = labels[train_list]
    test_y = labels[test_list]
    return(train_x, train_y, test_x, test_y)

# call the function to split the dataset
train_x, train_y, test_x, test_y = train_test_split(features, labels)

In [ ]:
train_y_df=pd.DataFrame()
train_y_df['labels']=train_y
train_y_df['labels'].value_counts()

In [ ]:
test_y_df=pd.DataFrame()
test_y_df['labels']=test_y
test_y_df['labels'].value_counts()

In [ ]:
train_x.shape

In [ ]:
train_y.shape

****

**Model**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_digits
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, min_samples_leaf= 3,random_state=42)

# Train the classifier
rf_classifier.fit(train_x, train_y)

# Make predictions on the test set
predictions = rf_classifier.predict(test_x)

# Generate the confusion matrix
cm = confusion_matrix(test_y, predictions)

target_names=[0,1,2,3,4,5,6]
# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, cmap="Blues", fmt="d", xticklabels=target_names, yticklabels=target_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
accuracy = accuracy_score(test_y, predictions)
precision = precision_score(test_y, predictions,average='macro')
recall = recall_score(test_y, predictions,average='macro')
f1 = f1_score(test_y, predictions,average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1:", f1)

In [ ]:
arr_prediction=rf_classifier.predict(features_chips)
arr_prediction.shape

In [ ]:
rf_prediction_arr=arr_prediction.reshape(780, 818)
rf_prediction_arr.shape

In [ ]:
fig, ax = plt.subplots(figsize=(795/80, 835/80))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.yticks(rotation=90, va='center')
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
cmap = mpl.colors.ListedColormap(['red','lightgreen','darkgreen','yellow','blue','lightyellow','gray'])
image = plt.imshow(rf_prediction_arr, cmap=cmap, extent=extent, interpolation='nearest')
divider = make_axes_locatable(ax)
cax = divider.new_vertical(size='5%', pad=0.5, pack_start=True)
fig.add_axes(cax)
cbar = plt.colorbar(image, orientation='horizontal', label='Alteration Type', cax=cax)
tick_locs = (np.arange(n_class_pos)+0.5)*(n_class_pos-1)/n_class_pos
cbar.set_ticks(tick_locs)
cbar.set_ticklabels(['Urban', 'Crop/Grass', 'Trees', 'Bare', 'Water','Coastal','Wetland'])
plt.grid(False)